In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

scratch_path <- Sys.getenv("SCRATCH")
.libPaths(file.path(scratch_path, "Rlocal4.3.2"))

In [ ]:
%%R
library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)

In [ ]:
%%R
OUT_DIR <- "../../../outputs/validation/"
N_FOLDS <- 10

vst.counts <- readRDS(paste(OUT_DIR, "vst_counts.Rds", sep = ""))

vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
GEO_model_validation_tissue_detail.vec <-
   readRDS(paste(OUT_DIR, "GEO_model_validation_tissue_vec.Rds", sep = ""))
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls.Rds", sep = ""))
rxns <- rxn2ensembls.nls %>% names()

In [ ]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [ ]:
%%R
toi_indices <- seq(1,length(GEO_model_validation_tissue_detail.vec))
# print(toi_indices)

# filter annotations
GEO_model_validation_tissue_detail_vec_tis_of_interest <- GEO_model_validation_tissue_detail.vec[toi_indices]
length(GEO_model_validation_tissue_detail_vec_tis_of_interest)

In [ ]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
length(vst.count.mtx.tis_of_interest)

In [ ]:
%%R
training_indices <-
   caret::createDataPartition(
      GEO_model_validation_tissue_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
length(training_indices)

In [ ]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of dataprint(vst.count.mtx.train)
print(length(vst.count.mtx.train))
print(length(vst.count.mtx.test))

In [ ]:
%%R
GEO_model_validation_tissue_detail.vec.train <-
   GEO_model_validation_tissue_detail_vec_tis_of_interest[training_indices]
GEO_model_validation_tissue_detail.vec.test <-
   GEO_model_validation_tissue_detail_vec_tis_of_interest[-training_indices]

print(length(GEO_model_validation_tissue_detail.vec.train))
print(length(GEO_model_validation_tissue_detail.vec.test))

saveRDS(GEO_model_validation_tissue_detail.vec.train,file=paste(OUT_DIR,"GEO_model_validation_tissue_detial_vec_train.Rds",sep=""))

In [ ]:
%%R
cv_fold_indices <- caret::createFolds(GEO_model_validation_tissue_detail.vec.train,
                                      k = N_FOLDS)
print(length(cv_fold_indices))
binary_GEO_model_validation_tissue_annotations <- unique(GEO_model_validation_tissue_detail.vec)
print(length(binary_GEO_model_validation_tissue_annotations))

full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [ ]:
%%R
print(length(rxns))

In [ ]:
%%R
n_rxns <- length(rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,".Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

In [ ]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()


In [ ]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train.Rds", sep = ""))

In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0
   
   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      GEO_model_validation_tissue_detail.vec.train.cv_train <-
         GEO_model_validation_tissue_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      GEO_model_validation_tissue_detail.vec.train.cv_test <-
         GEO_model_validation_tissue_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_GEO_model_validation_tissue_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_GEO_model_validation_tissue_annotations) {
         binary_GEO_model_validation_tissue_detail_vec.test.cv_test_list[[tissue_annotation]] <-
            (GEO_model_validation_tissue_detail.vec.train.cv_test == tissue_annotation)
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = GEO_model_validation_tissue_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            GEO_model_validation_tissue_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_GEO_model_validation_tissue_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         
         tab <- table(cur_rxn_knn_calls,
                      binary_GEO_model_validation_tissue_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_GEO_model_validation_tissue_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

In [ ]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls.Rds", sep = ""))

In [ ]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df.Rds", sep = ""))

In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = GEO_model_validation_tissue_detail.vec.train
)
ggplot(pca.d) +
   geom_point(aes(x = PC1, y = PC2, colour = Section)) +
   theme_bw()
p <- plot_ly(
   x = pca.d$PC1,
   y = pca.d$PC2,
   z = pca.d$PC3,
   type = "scatter3d",
   mode = "markers",
   color = pca.d$Section,
   size = 1
)
saveWidget(p, paste0(OUT_DIR, "plotly_chart2.html"), selfcontained = TRUE)